In [1]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from copy import deepcopy

from selfea import * 

In [2]:
DataInit = InitData()

start reading data
read train
read test
finished


In [3]:
train = DataInit.get_train(False)
test = DataInit.get_test(False)

In [4]:
TARGET = "is_female"

In [5]:
X_new = DataInit.get_train(True)
Y_new = DataInit.get_labels()
T_new = DataInit.get_test(True)

In [6]:
x_n_train, x_n_test, y_n_train, y_n_test = train_test_split(X_new, Y_new, test_size=0.4, random_state=42)

In [7]:
def classify(clf,x_n_train, y_n_train, x_n_test,y_n_test,test, useall = False):
    if useall:
        clf.fit(x_n_train, y_n_train)
        test_n = test.reindex(columns = x_n_train.columns)
    else:
        clf.fit(x_n_train, y_n_train)

        y_pred = clf.predict(x_n_test)

        cm = confusion_matrix(y_n_test, y_pred)
        print (cm)

        y_prob = clf.predict_proba(x_n_test)
        print(metrics.roc_auc_score(y_n_test, y_prob[:,1]))

        test_n = test.reindex(columns = x_n_test.columns)

    y_pred_f1 = clf.predict(test_n)
    y_prob = clf.predict_proba(test_n)
    return y_prob

# RandomForestClassifier

In [8]:
Ntree = 700
clf = RandomForestClassifier(n_estimators=Ntree,random_state=14, max_features = 0.6)
y_prob_rfc = classify(clf,x_n_train, y_n_train, x_n_test,y_n_test,test, useall = False)

[[3059  336]
 [ 409 3498]]
0.960544779526


In [239]:
Ntree = 500
clf = RandomForestClassifier(n_estimators=Ntree,random_state=1234)
# clf.fit(x_n_train, y_n_train)
clf.fit(X_new, Y_new)


test_n = test.reindex(columns = X_new.columns)

y_pred_f1 = clf.predict(test_n)
y_prob = y_prob_rfc = classify(clf,x_n_train, y_n_train, x_n_test,y_n_test,test, useall = True)

print(y_prob)

[[ 0.028  0.972]
 [ 0.918  0.082]
 [ 0.01   0.99 ]
 ..., 
 [ 1.     0.   ]
 [ 0.498  0.502]
 [ 0.13   0.87 ]]


# SVM

In [ ]:
# from sklearn import svm

# x_n_train, x_n_test, y_n_train, y_n_test = train_test_split(X_new, Y_new, test_size=0.4, random_state=42)

clf = svm.SVC(kernel = "linear",probability = True)
clf.fit(x_n_train, y_n_train) 

# KNN

In [23]:
# from sklearn.neighbors import KNeighborsClassifier
# x_n_train, x_n_test, y_n_train, y_n_test = train_test_split(X_new, Y_new, test_size=0.4, random_state=42)
clf = KNeighborsClassifier(n_neighbors=10)
clf.fit(x_n_train, y_n_train) 


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [24]:

y_pred = clf.predict(x_n_test)

cm = confusion_matrix(y_n_test, y_pred)
print (cm)

y_prob = clf.predict_proba(x_n_test)
print(metrics.roc_auc_score(y_n_test, y_prob[:,1]))

test_n = test.reindex(columns = x_n_test.columns)

y_pred_f1 = clf.predict(test_n)
y_prob = clf.predict_proba(test_n)


print(y_prob)

[[1892 1503]
 [1622 2285]]
0.60000701132
[[ 0.4  0.6]
 [ 0.8  0.2]
 [ 0.5  0.5]
 ..., 
 [ 0.5  0.5]
 [ 0.7  0.3]
 [ 0.7  0.3]]


# Neuronetworks

lst = []
acc = 0
for i in (train.dtypes):
    if str(i) == "object":
        lst.append(acc)
    acc = acc+1
    
lst2 = []
for i in lst:
    lst2.append(train.dtypes.keys()[i])


In [38]:
train = pd.read_csv("train.csv",low_memory=False)
test = pd.read_csv("test.csv",low_memory=False)

In [39]:
df = deepcopy(train)
category_cols = df.select_dtypes(exclude=[np.number]).columns.tolist() + test.select_dtypes(exclude=[np.number]).columns.tolist()
# print(len(category_cols))

In [40]:

# drop_lst = category_cols + ["is_female","train_id"]
Y_new = df.is_female
X_new = df.drop(["is_female","train_id"], axis = 1)

for header in category_cols:
    X_new[header] = X_new[header].astype('category').cat.codes
    X_new[header] = pd.to_numeric(X_new[header])
    test[header] = test[header].astype('category').cat.codes
    test[header] = pd.to_numeric(test[header])

In [41]:
X_new.fillna("3",inplace = True)
test.fillna("3",inplace = True)
scaler = StandardScaler()
scaler.fit(X_new)
StandardScaler(copy=True, with_mean=True, with_std=True)
X_new  = scaler.transform(X_new)

In [42]:
test = scaler.transform(test)
x_n_train, x_n_test, y_n_train, y_n_test = train_test_split(X_new, Y_new, test_size=0.4, random_state=42)

ValueError: operands could not be broadcast together with shapes (27285,1234) (1233,) (27285,1234) 

In [64]:
# x_n_train, x_n_test, y_n_train, y_n_test = train_test_split(X_new, Y_new, test_size=0.4, random_state=42)
clf = MLPClassifier(solver='adam')
clf.fit(x_n_train, y_n_train)

y_pred = clf.predict(x_n_test)

cm = confusion_matrix(y_n_test, y_pred)
print (cm)

y_prob = clf.predict_proba(x_n_test)
print(metrics.roc_auc_score(y_n_test, y_prob[:,1]))

test_n = test.reindex(columns = x_n_test.columns)

y_pred_f1 = clf.predict(test_n)
y_prob = clf.predict_proba(test_n)


print(y_prob)

[[3395    0]
 [3907    0]]
0.536752092935
[[  1.00000000e+000   0.00000000e+000]
 [  1.00000000e+000   8.67956728e-279]
 [  1.00000000e+000   0.00000000e+000]
 ..., 
 [  1.00000000e+000   2.88097250e-128]
 [  1.00000000e+000   7.02877052e-213]
 [  1.00000000e+000   3.46772627e-228]]


# Write submission file

In [240]:
import csv

with open("submit_all.csv",'w') as f:
    fieldnames = ["test_id","is_female"]
    wri = csv.DictWriter(f, delimiter=',',fieldnames= fieldnames)
    acc = 0
    wri.writeheader()
    for i in y_prob:
        wri.writerow({"test_id":acc, "is_female": i[1]})
        acc = acc+1

In [241]:
t = pd.read_csv("submit2.csv")
t2 = pd.read_csv("submit_all.csv")
print(t.head())
print(t2.head())

   test_id  is_female
0        0      0.978
1        1      0.130
2        2      0.990
3        3      0.986
4        4      0.514
   test_id  is_female
0        0      0.972
1        1      0.082
2        2      0.990
3        3      0.974
4        4      0.530
